In [1]:
import torch
import pandas as pd
import string
from tqdm import tqdm
import numpy as np
import pickle

c:\Users\Wen2Tee5\Desktop\Postgraduate\torchPlayGround\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# HyperParameter

In [2]:
EPOCH = 20
DATA_NUM = 200

In [3]:
stop_words = []
with open('./NLPUtils/english.txt', encoding='utf-8') as f:
    stop_words = [stop_word.replace('\n','') for stop_word in f.readlines()]

In [4]:
file_path = './basic_dataset/IMDB/IMDB Dataset.csv'

df = pd.read_csv(file_path, encoding='utf-8')

In [5]:
sentences = [sentence.lower() for sentence in df['review'][:DATA_NUM]]

In [6]:
print(f'Raw text Data -> {sentences[0]}')

Raw text Data -> one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.<br /><br />the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.<br /><br />it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />i would say the main appeal of the show 

In [7]:
for i in tqdm(range(len(sentences))):
    sentences[i] = sentences[i].translate(str.maketrans('', '', string.punctuation))

100%|██████████| 200/200 [00:00<00:00, 100007.25it/s]


In [8]:
def tokenize(sentences) -> list:
    tokens = [sentence.split() for sentence in sentences]
    return tokens

In [9]:
tokens = tokenize(sentences)

In [10]:
for i in tqdm(range(len(tokens))):
    for stop_word in stop_words:
        tokens[i] = list(filter(stop_word.__ne__, tokens[i]))

100%|██████████| 200/200 [00:00<00:00, 244.13it/s]


In [11]:
print(f'Preprocess Data -> {tokens[0]}')

Preprocess Data -> ['reviewers', 'mentioned', 'watching', '1', 'oz', 'episode', 'youll', 'hooked', 'happened', 'mebr', 'br', 'struck', 'oz', 'brutality', 'unflinching', 'scenes', 'violence', 'set', 'word', 'trust', 'faint', 'hearted', 'timid', 'pulls', 'punches', 'drugs', 'sex', 'violence', 'hardcore', 'classic', 'wordbr', 'br', 'called', 'oz', 'nickname', 'oswald', 'maximum', 'security', 'penitentary', 'focuses', 'emerald', 'city', 'experimental', 'prison', 'cells', 'glass', 'fronts', 'face', 'inwards', 'privacy', 'high', 'agenda', 'em', 'city', 'manyaryans', 'muslims', 'gangstas', 'latinos', 'christians', 'italians', 'irish', 'moreso', 'scuffles', 'death', 'stares', 'dodgy', 'dealings', 'shady', 'agreements', 'awaybr', 'br', 'main', 'appeal', 'fact', 'wouldnt', 'forget', 'pretty', 'pictures', 'painted', 'mainstream', 'audiences', 'forget', 'charm', 'forget', 'romanceoz', 'doesnt', 'mess', 'episode', 'struck', 'nasty', 'surreal', 'ready', 'watched', 'developed', 'taste', 'oz', 'accust

In [12]:
voc = []

In [13]:
for sentence in tqdm(tokens):
     for word in sentence:
        if word not in voc:
            voc.append(word)

100%|██████████| 200/200 [00:00<00:00, 606.95it/s]


In [14]:
print(f'The size of voc {len(voc)}')
print(f'The first word of voc is {voc[0]}')

The size of voc 7595
The first word of voc is reviewers


In [15]:
with open('./basic_dataset/IMDB/voc.data', 'wb') as f:
    pickle.dump(voc, f)

In [16]:
word2idx = {w:idx for (idx, w) in enumerate(voc)}
inx2word = {idx:w for (idx, w) in enumerate(voc)}

In [17]:
window_size = 2
idx_pair = set()
for sentence in tqdm(tokens):
    indices = [word2idx[word] for word in sentence]
    for idx in range(len(indices)):
        if idx + window_size - 1 < len(indices):
            window = indices[idx : idx + window_size]
            for x_word in window:
                for y_word in window:
                    if x_word != y_word:
                        idx_pair.add((x_word, y_word))

100%|██████████| 200/200 [00:00<00:00, 13329.85it/s]


In [18]:
one_hot_vector = np.zeros(shape=(len(voc), len(voc)))

In [19]:
for idx, vector in enumerate(one_hot_vector):
    vector[idx] = 1.

In [20]:
x_data = []
y_data = []
print(f'Converting to One-Hot Vector')
for x_word_idx, y_word_idx in tqdm(idx_pair):
    x_data.append(one_hot_vector[x_word_idx])
    y_data.append(one_hot_vector[y_word_idx])
print(f'Done!')

Converting to One-Hot Vector


100%|██████████| 35864/35864 [00:00<00:00, 1707254.86it/s]

Done!


In [21]:
assert len(x_data) == len(y_data), 'ERROR len of x != len of y'

In [22]:
x = np.array(x_data)
y = np.array(y_data)

In [23]:
print(x.shape)
print(y.shape)

(35864, 7595)
(35864, 7595)


In [24]:
np.save('./basic_dataset/IMDB/X_DATA.npy', x)
np.save('./basic_dataset/IMDB/Y_DATA.npy', y)

# 50000筆資料如果要用one-hot vector表示需要240GB的大小